# Text Analytics Coursework -- Tasks 1.1 to 1.3

# Instructions:

This notebook is forms the first part of your coursework assignment for Text Analytics in Spring 2025. You will need to read the instructions below and complete numbered tasks indicated by "TASK n". To complete the tasks, you will write code or explanations between the comments "#WRITE YOUR ANSWER HERE" and "#END OF ANSWER". For example:

TASK 0: Complete the function below to output "hello world". 


In [1]:
def demo_fun():
    # WRITE YOUR ANSWER HERE
    print("hello world")
    # END OF ANSWER

### DO NOT MODIFY
demo_fun()
###

hello world


There is also some code in the cell that should not be modified. This code saves your results to file in the correct format, which is necessary for us to be able to mark your answers. Before you submit your notebook, please make sure this code has not been modified, then restart your kernel, clear all cell outputs, run all of your code once again, then save the notebook. 

Please note:
  * The notebook you upload must include all the saved cell output after running all cells.
  * The notebook code must be complete so that it reproduces all your output when we run it. 
  * For this coursework, we recommend that you use your virtual environment that you created for the labs. The packages you need are: numpy, scipy, nltk, pytorch, transformers and datasets (from HuggingFace), pandas, matplotlib and scikit-learn. 

## Marking guidelines:
1. This notebook is worth 32% of the marks for the Text Analytics assignment.
1. The number of marks for each task is shown alongside the task.
1. We will evaluate the output of your code after running it, and marks will be awarded based on how well the output matches the task's instructions. 
1. We will give partial marks for incomplete or partially correct answers. 
1. We do not give additional marks for code style or comments, but clear code will help us to understand what you have done so that we can award partial marks where necessary. 
1. Unless the task asks you to implement something from scratch, there is no penalty for using software libraries in your implementation.

## Support:

The main source of support will be during the lab sessions. The TAs and lecturers will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the assessed tasks, they can only answer clarifying questions about what you have to do. Please email Edwin if you have any other queries edwin.simpson@bristol.ac.uk and/or post your query to the Teams channel for this unit.

## Deadline:

The notebook must be submitted along with the second notebook on Blackboard before **Monday 28th April at 13.00**. 

## Submission:

For this part of the assignment, please zip up the folder containing this file and the 'outputs' directory, containing the output from this notebook as .csv files. Please name your notebook file like this:
   * Name this notebook 'text_analytics_part1_\<student number\>.ipynb'. Replace '\<student number\>' with your student number, which consists only of digits beginning with '2'. 
   * We mark anonymously, so please don't include your name in the notebook.

You can submit the file on Blackboard to the submission point "Text Analytics Part 1 Notebook". Remember that the assignment also has parts 2 and 3, described in the PDF file on Blackboard.

# Setup: random seeds

Each student will work with slightly different data splits and model weights, which will be determined by setting your 'random seed'. 
We will check that your results come from using your random seed. Please set the seed in the cell below by changing the value of 'my_student_number' to your own student number (not your username, the number you can see on eVision that contains only digits). 

Using the correct seed ensures that your results are reproducible when we rerun your notebook.

In [2]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python's built-in random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable benchmark mode for reproducibility

### SET YOUR SEED TO YOUR STUDENT NUMBER HERE
my_student_number = 2632994
set_seed(my_student_number)

# Setup: loading the data

Let's make a folder to save the output of your work:

In [3]:
import os
import pandas as pd

os.makedirs('./outputs', exist_ok=True)

Now, let's load some more packages we will need later:

In [4]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the Emotion dataset
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split 


The dataset classifies paragraphs taken from corporate disclosures that discuss climate-related issues. It classifiers them into "risk" (0), "neutral" (1) or "opportunity" (2) representing the sentiment of the paragraph.

First we need to load the data. The data is already split into train, validation and test. The _validation_ set (also called 'development' set or 'devset') can be used to compute performance of your model when tuning hyperparameters, optimising combinations of features, or looking at the errors your model makes before improving it. This allows you to hold out the test set (i.e., not to look at it at all when developing your method) to give a fair evaluation of the model and how well it generalises to new examples. This avoids tuning the model to specific examples in the test set. An alternative approach to validation is to not use a single fixed validation set, but instead use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

In [5]:
cache_dir = "./data_cache"

# load the original training set from HuggingFace
train_dataset = load_dataset(
    "climatebert/climate_sentiment",
    split="train",
    cache_dir=cache_dir,
)

# we're going to create a new validation set by splitting the data
dataset_splits = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_splits["train"]
val_dataset = dataset_splits["test"]

train_texts = np.array(train_dataset["text"])
val_texts = np.array(val_dataset["text"])

train_labels = np.array(train_dataset["label"])
val_labels = np.array(val_dataset["label"])

print(f"Training dataset with {len(train_texts)} instances loaded")
print(f"Development/validation dataset with {len(val_texts)} instances loaded")

### DO NOT MODIFY
# save gold labels to file
pd.DataFrame(val_labels).to_csv('./outputs/val_labels.csv')

Training dataset with 800 instances loaded
Development/validation dataset with 200 instances loaded


In this notebook, you're going to build three different classifiers for this dataset, then compare how they work, and analyse the results. We are going to start by implementing a naïve Bayes classifier from scratch. 

We are going to begin by initialising some useful variables and doing some very simple pre-processing using CountVectorizer.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

num_classes = 3

def preprocess(train_texts):
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess(train_texts)

/opt/anaconda3/envs/r_env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## TASK 1.1a

Complete the function below to compute the class priors, $p(y_n = c)$ for each class label $c$, where $y_n$ is the class label of document $n$. Do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the class priors as a list or Numpy array containing the probabilities. You do not need to apply any smoothing or regularisation.    (3 marks)

In [7]:
def compute_class_priors(texts, labels):
    priors = np.zeros(num_classes)

    ### WRITE YOUR ANSWER HERE
    for c in range(num_classes):
        # Compute probability for each class
        priors[c] = np.sum(labels == c) / len(labels)  

    ### END OF ANSWER
    return priors


class_priors = compute_class_priors(train_texts, train_labels)
print(class_priors)

### DO NOT MODIFY
pd.DataFrame(class_priors).to_csv('./outputs/11a_class_priors.csv')

[0.34375 0.41    0.24625]


## TASK 1.1b

Complete the function below to extract n-gram features from the text, then compute the liklihood $p(x_{ni} = w | y_n = c)$ that the $i$ th n-gram in document $n$ is $w$, given that the class of $n$ is $c$. Again, do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the likelihoods as a 2D Numpy array containing probabilities. You should apply smoothing by adding counts of +1 to the counts of each feature.  (3 marks)

In [8]:
def compute_feature_likelihoods(X, labels):

    likelihoods = np.ones((num_features, num_classes))  # a 2D numpy array where you can store the likelihoods. Note that all values are initialised to one.

    ### WRITE YOUR ANSWER HERE
    for c in range(num_classes):
        class_indices = np.where(labels == c)[0]  # Get indices of documents belonging to class c
        likelihoods[:, c] += np.sum(X[class_indices], axis=0)  # Sum feature counts for class c
    # Normalize to get probabilities
    # likelihoods /= likelihoods.sum(axis=0)
    likelihoods = likelihoods / likelihoods.sum(axis=0)
    ### END OF ANSWER

    ### END OF ANSWER
    return likelihoods

likelihoods = compute_feature_likelihoods(X, train_labels)

### DO NOT MODIFY
pd.DataFrame(likelihoods).to_csv('./outputs/11b_likelihoods.csv')

Now, we are going to use the code in the next cell to compute the log probabilities of each class for each text in the validation set. This code will use the previous functions you implemented, compute_class_priors and compute_feature_likelihoods. The log probabilities will be stored in the 'predictions' array.

In [9]:
from scipy.special import logsumexp


def NB_classify(class_priors, likelihoods, X_val):

    predictions = np.zeros((X_val.shape[0], num_classes))  # an empty numpy array to store the predictions in

    sum_of_log_likelihoods = X_val.dot(np.log(likelihoods))
    log_joint_prob = sum_of_log_likelihoods + np.log(class_priors)[None, :]
    for n, doc in enumerate(X_val):
        predictions[n, :] = log_joint_prob[n]
        predictions[n, :] -= logsumexp(predictions[n, :])
    return predictions

predictions = NB_classify(class_priors, likelihoods, X_val)
print(predictions)

[[-3.98892814e-01 -1.11188790e+00 -2.02379806e+01]
 [-1.38406811e+01 -1.33262136e+01 -2.60631600e-06]
 [-3.97190793e+01 -6.04778462e-03 -5.11108562e+00]
 [-4.61384688e+00 -3.66729971e-01 -1.21372136e+00]
 [-4.25938695e+00 -8.18498772e-01 -6.07381133e-01]
 [-2.96724506e+00 -3.37591268e+00 -8.95218841e-02]
 [-3.45525977e-04 -7.97070158e+00 -1.73307556e+01]
 [-7.11687650e+00 -1.68369116e+00 -2.06407638e-01]
 [-1.73236783e+01 -2.99639851e-08 -2.51292235e+01]
 [ 0.00000000e+00 -6.32668428e+01 -6.65654851e+01]
 [-1.97596423e+00 -6.87834136e-01 -1.02524428e+00]
 [-5.05476770e+00 -6.39956832e-03 -1.50662066e+01]
 [-1.41007373e-01 -2.02861847e+00 -2.10127077e+01]
 [-9.97666464e-05 -1.03849568e+01 -9.58332004e+00]
 [-2.97274028e-09 -1.96707617e+01 -2.29496647e+01]
 [-1.02602371e-10 -2.30009032e+01 -3.01825900e+01]
 [-1.95730860e-06 -1.31439448e+01 -2.56748833e+01]
 [-1.92583081e+01 -2.52695713e-08 -1.76814999e+01]
 [ 0.00000000e+00 -6.33815386e+01 -8.19186209e+01]
 [-1.48929757e-11 -2.49298281e+

Use the 'predictions' array above to compute and print the accuracy of the classifier on the validation set.   

In [10]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, np.argmax(predictions, axis=1))

0.77

## TASK 1.1c

The simplicty of naïve Bayes means that we can quite easily interpret the model. In the code above, we used the functions you implemented, compute_feature_likelihoods and compute_class_priors, to train an NB classifier with our training set. Given this classifier, which are the five n-gram features that most strongly indicate that the document belongs to class 0? Store these features in the 'top_features' list below.    (4 marks)

In [11]:
top_features = []

### WRITE YOUR ANSWER HERE
class_0_likelihoods = likelihoods[:, 0]  # Extract likelihoods for class 0
top_indices = np.argsort(class_0_likelihoods)[-5:]  # Get indices of top 5 features
top_features = [vectorizer.get_feature_names_out()[i] for i in top_indices]  # Retrieve feature names



### END OF ANSWER

### DO NOT MODIFY
print(top_features)
pd.DataFrame(top_features).to_csv('./outputs/11c_top_feats.csv')
###

['to the', 'in the', ', and', 'of the', 'climate change']


Up to this point, the classifier used bigrams features extracted using CountVectorizer with the wordnet tokenizer. 

## TASK 1.1d

Your task is to improve the naïve Bayes classifier by changing the preprocessing or features only. It is up to you to decide how many changes are needed to improve the classifier -- a single change may be enough to achieve a good result (and maximum marks) and you should only include steps that help performance. Complete the 'preprocess_improved' function below, and run the cell to compute accuracy of the improved classifier on the validation set.     (3 marks)

In [12]:
def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE

    vectorizer = CountVectorizer(ngram_range=(1,2), token_pattern=None, tokenizer=word_tokenize, stop_words='english', max_features=10000)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

0.8


## TASK 1.2

Below is an implementation of a neural network classifier that we can apply to the same dataset. However, there are some mistakes in the code and some poor choices in the choice of parameters and architecture. Your task is to fix the errors, make better parameter choices, and improve the model's performance. **Modify the code within the next cell** to improve the neural network classifier, then run it and compute its accuracy using the code in the cell after that.   (8 marks)

In [13]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE: MODIFY THE CODE WITHIN THIS CELL

# Importing nn module from the Pytorch library, which is used for building neural networks in Pytorch.
from torch import nn
# Importing optim module from the Pytorch library, which is used for optimization algorithms in Pytorch (Adams optimizer).
from torch import optim
# This line imports the AutoTokenizer class from the transformers library (automatically load the appropriate tokenizer for a given pre-trained model). 
from transformers import AutoTokenizer
# Importing the DataLoader class from the torch.utils.data module, which is used for loading datasets in Pytorch.
from torch.utils.data import DataLoader
# Importing the accuracy_score function from sklearn.metrics, which is used to calculate the accuracy of predictions.
import torch.nn.functional as F


# Loading the tokenizer for the ClimateBERT model, which is fine tuned for climate-related text.
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

# Setting up hyperparameters for the model.
# num_classes = 3, hence 3 nodes in the output layer. 
sequence_length = 120
batch_size = 32
embedding_dim = 256
hidden_size = 512
num_epochs = 7
learning_rate = 5e-4

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=sequence_length,
        return_tensors="pt"
    )

# Applying the tokenize_function to the training dataset and setting the format for PyTorch.
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
# Creating a DataLoader for the training set, which will load batches of data during training.
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

# Applying the tokenize_function to the validation dataset and setting the format for PyTorch
tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
# Creating a DataLoader for the validation set, which will load batches of data during validation.
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False)

class FFTextClassifier(nn.Module):

    def __init__(self, vocab_size, sequence_length, embedding_size, hidden_size, num_classes):
        super(FFTextClassifier, self).__init__()

        self.embedding_size = embedding_size
        self.sequence_length = sequence_length
        # Embedding layer, converts input token indices to dense vectors of fixed size (embedding_size).
        self.embedding_layer = nn.Embedding(vocab_size, embedding_size)
        self.conv1 = nn.Conv1d(embedding_size, hidden_size, kernel_size=3, padding=1) # Here, kernel_size and padding ensure that the output has the same lenght as the input.
        self.activation1 = nn.ReLU()
        # Reduces the sequence length by taking the maximum value over the entire sequence. 
        # Using AdaptiveMaxPool1d with output size 1, which reduces the sequence dimension to a single value per feature.
        self.pool = nn.AdaptiveMaxPool1d(1)
        # The dropout rate is set to 50% i.e half of the units in the previous layer are randomly dropped out.
        self.dropout = nn.Dropout(0.5)
        # Fully connected layer, linear layer that reduces the output size from hidden_size to hidden_size // 2.
        self.hidden_layer2 = nn.Linear(hidden_size, hidden_size // 2)
        self.activation2 = nn.ReLU()
        self.output_layer = nn.Linear(hidden_size // 2, num_classes)

    def forward(self, input_ids, attention_mask=None):
        # Passing the input through the embedding layer, converting token IDs into word embeddings.
        embedded_words = self.embedding_layer(input_ids)
        # Transposes the embedded words from shape (batch_size, seq_len, embedding_size) to (batch_size, embedding_size, seq_len), required for the 1D convolution.
        embedded_words = embedded_words.transpose(1, 2)
        # Applying the convolutional layer, extracting features from the sqeuence (n-grams).
        x = self.conv1(embedded_words)
        x = self.activation1(x)
        # Applying AdaptiveMaxPool1d to reduce the sequence length to 1 (reducing the tensor from 3D to 2D).
        x = self.pool(x)
        # After pooling, the tensor has shape (batch_size, hidden_size, 1), so we remove the last dimension.
        x = x.squeeze(2)
        # Applying dropout to prevent overfitting, works by randomly setting some of the activations to zero during training.
        x = self.dropout(x)

        x = self.hidden_layer2(x)
        x = self.activation2(x)
        # Passing the output through the final output layer, which has num_classes = 3 nodes, layer produces the final logits for each class.
        output = self.output_layer(x)
        return output

    def run_training(self, train_dataloader, dev_dataloader):

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        dev_losses = []

        for e in range(num_epochs):
            train_losses = []
            self.train()

            for i, batch in enumerate(train_dataloader):
                optimizer.zero_grad()
                input_ids = batch['input_ids']
                output = self(input_ids)
                batch_loss = loss_fn(output, batch['label'])
                batch_loss.backward()
                optimizer.step()
                train_losses.append(batch_loss.item())

            print(f"Epoch: {e + 1}/{num_epochs}, Training Loss: {np.mean(train_losses):.4f}")

            self.eval()
            dev_losses_epoch = []

            for dev_batch in dev_dataloader:
                input_ids = dev_batch['input_ids']
                dev_output = self(input_ids)
                dev_loss = loss_fn(dev_output, dev_batch['label'])
                dev_losses_epoch.append(dev_loss.item())

            dev_losses.append(np.mean(dev_losses_epoch))
            print(f"Epoch: {e + 1}/{num_epochs}, Validation Loss: {dev_losses[-1]:.4f}")

        return dev_losses

def predict_nn(trained_model, data_loader):
    trained_model.eval()
    pred_labs = []

    for batch in data_loader:
        input_ids = batch['input_ids']
        test_output = trained_model(input_ids)
        predicted_labels = test_output.argmax(1)
        pred_labs.extend(predicted_labels.tolist())

    return pred_labs

vocab_size = max(tokenizer.get_vocab().values()) + 1 # Original line of code
nn_classifier_model = FFTextClassifier(vocab_size, sequence_length, embedding_dim, hidden_size, num_classes)
dev_losses = nn_classifier_model.run_training(train_loader, val_loader)

predictions_nn = predict_nn(nn_classifier_model, val_loader)

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)
###

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch: 1/7, Training Loss: 1.1373
Epoch: 1/7, Validation Loss: 1.0271
Epoch: 2/7, Training Loss: 0.9630
Epoch: 2/7, Validation Loss: 0.9010
Epoch: 3/7, Training Loss: 0.7837
Epoch: 3/7, Validation Loss: 0.7733
Epoch: 4/7, Training Loss: 0.5994
Epoch: 4/7, Validation Loss: 0.6441
Epoch: 5/7, Training Loss: 0.4747
Epoch: 5/7, Validation Loss: 0.5775
Epoch: 6/7, Training Loss: 0.3204
Epoch: 6/7, Validation Loss: 0.5420
Epoch: 7/7, Training Loss: 0.3097
Epoch: 7/7, Validation Loss: 0.5250
0.75


We now explore the use of transformers for building a text classifier. First, let's look at how the process a document. We'll chose one at random from the training set:

In [14]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]

## TASK 1.3a

Use the HuggingFace transformers library to load the pretrained BERT model "prajjwal1/bert-tiny". Obtain a document embedding for the chosen document given above. Comment your code to explain how it obtains a representation of the document.    (3 marks)

In [15]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE

from transformers import AutoTokenizer, BertModel  
import torch

# Loading the pretrained BERT model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = BertModel.from_pretrained('prajjwal1/bert-tiny')  
# Setting the maximum sequence length to 128.
max_length = 128  
# Tokenize the chosen document.
encoded_input = tokenizer(chosen_document, padding=True, max_length=max_length, truncation=True, return_tensors='pt')

# Get the models output.
with torch.no_grad():  
    output = model(**encoded_input)  

# The output contains various information. For document embedding, we can use the [CLS] token's output.
# The [CLS] token is the first token in the sequence, and its final hidden state is often used as a representation of the whole sequence.
doc_emb = output.last_hidden_state[:, 0, :].numpy()  # Shape: (batch_size, hidden_size) [cite: text_analytics_part1.ipynb]

### END OF ANSWER

### DO NOT MODIFY
import pandas as pd

pd.DataFrame(doc_emb).to_csv('./outputs/13a_sen_emb.csv')
print(doc_emb)
###

[[ 0.68653965  1.4661262  -4.0584407  -0.41935784  0.40189984  0.11576049
   0.36171576  1.5808071   0.09927985  1.2442403   0.6996219   1.1368225
   0.01021597 -0.35720953  1.9377443  -0.30223125 -1.005615   -1.1137446
  -0.4699459   0.9816291  -0.7223004   0.76492417  2.1274219   0.73005134
   2.3007264  -0.37367326 -0.51506144 -0.6647792  -0.39258623  0.83951664
   1.3212192  -0.83370197 -1.0029147  -0.19491567  0.59751004 -1.3736687
  -1.0464209   0.9198849  -2.1951315  -0.7254908  -0.43582526  0.07712255
  -0.12450959 -1.9939997   0.40623534 -1.7777809  -0.1449757  -0.4690484
  -0.5414165   0.9283023   0.9859487   1.3369174  -0.68394727  0.12364272
  -1.1535665  -0.408989   -1.4303138  -1.4013221   0.43551823  2.7341714
  -0.89353186 -0.44960582 -2.1531515  -2.0251927  -1.3466438   0.5065092
   0.9202897   0.5219299  -2.4669359  -1.4244568  -1.1668272   0.02559411
   1.0330344  -0.16518018 -0.41084278  0.51703537  0.50360644 -0.4051518
   0.9830454   0.63797486 -0.6664931  -0.8671

## TASK 1.3b

Using the same document embeddings method as the previous task, find the most similar document to the 'chosen_document' from within the validation set (from the 'val_texts' object). Use a standard similarity metric that considers the direction but not the magnitude of the embedding vectors. Use the same model as in task 1.8.  (2 marks)

In [16]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
# Install scikit-learn if not already installed
%pip install scikit-learn

# Importing cosine_similarity from sklearn.metrics.pairwise for calculating cosine similarity between vectors, to measure how similar the vectors are.
from sklearn.metrics.pairwise import cosine_similarity

# Getting the embedding for the chosen document
chosen_document_embedding = doc_emb

# Finding the most similar document in val_texts to chosen_document.
highest_similarity = -1.0
most_similar_doc = ""

# Iterating through each document in the validation set.
for val_text in val_texts:
    # Tokenize the validation document and get its embedding.
    encoded_input_val = tokenizer(val_text, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    with torch.no_grad():
        output_val = model(**encoded_input_val)
    val_text_embedding = output_val.last_hidden_state[:, 0, :].numpy()

    # Calculating cosine similarity
    similarity = cosine_similarity(chosen_document_embedding, val_text_embedding)[0][0]

    # Updating most similar document if current similarity is higher.
    if similarity > highest_similarity:
        highest_similarity = similarity
        most_similar_doc = val_text

### END OF ANSWER

### DO NOT MODIFY
import pandas as pd
pd.DataFrame([chosen_document, most_similar_doc]).to_csv("./outputs/13b_most_similar.csv")
print(chosen_document)
print(most_similar_doc)
###

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
We made three key commitments to deal with key land, urban and industry shifts. These included launching a partnership with ClimateWorks Australia to develop sustainable agricultural metrics to improve natural asset management, investing $2 billion in affordable housing and investing $2 billion in the emerging technology sector to spur innovation by 2025.
$200 billion in financing to sustainable businesses and projects by 2030, with more than 50 percent focused on clean technology and renewable energy transactions to help accelerate the transition to a low-carbon economy. This commitment demonstrates how our products and services, operations and culture, and philanthropy can be harnessed toward a single goal. As an example,


## TASK 1.3c

Implement a classifier based on the same pretrained transformer model, "prajjwal1/bert-tiny". Evaluate your model's performance on the validation set. Use an 'auto class' from HuggingFace to build your classifier (see https://huggingface.co/docs/transformers/model_doc/auto).   (6 marks)

In [17]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import torch

# Load tokenizer and model
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Tokenization function
def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Prepare HuggingFace datasets
train_dataset_hf = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset_hf = Dataset.from_dict({"text": val_texts, "label": val_labels})

train_dataset_hf = train_dataset_hf.map(tokenize_fn, batched=True)
val_dataset_hf = val_dataset_hf.map(tokenize_fn, batched=True)

# Set format for PyTorch
train_dataset_hf.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset_hf.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Training arguments (improved)
training_args = TrainingArguments(
    output_dir="./outputs/tinybert_classifier",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=50,
    logging_dir="./outputs/logs",
    logging_strategy="epoch",
    save_strategy="no",
    seed=my_student_number
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=val_dataset_hf,
)

# Train the model
trainer.train()

# Predict on validation set
predictions = trainer.predict(val_dataset_hf)
predictions_bert = np.argmax(predictions.predictions, axis=1).tolist()


### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_tinybert = accuracy_score(val_dataset["label"], predictions_bert)
print(accuracy_tinybert)
### 


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/opt/anaconda3/envs/r_env/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.111300,1.077486
2,1.047000,0.998826
3,0.969900,0.921870
4,0.901100,0.850928
5,0.837600,0.806203
6,0.790200,0.771311
7,0.759000,0.745940
8,0.735100,0.733767
9,0.721300,0.724713
10,0.717500,0.722363


0.785
